In [15]:
import pandas as pd
import numpy as np
import duckdb
import os, pathlib
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
def sqldf(query):
    return duckdb.query(query).to_df()

## Twitter Drug Usage Dataset
이 데이터셋은, 트위터에서 약물을 사용한 트윗을 구분함.

requests 라이브러리를 사용하여, 해당 트윗의 내용을 가져올 필요가 있음

In [23]:
main_data = pd.read_csv("dataset/(Dataset 4) Drug Use Tweets/Publish dataset/Main_data.csv")
other_data = pd.read_csv("dataset/(Dataset 4) Drug Use Tweets/Publish dataset/Other_data.csv")

In [25]:
main_data.head()

,url,label
0,https://twitter.com/youroomate_/status/1520027...,F
1,https://twitter.com/Mr_McStallion/status/15200...,T
2,https://twitter.com/Admiral_Nickels/status/151...,T
3,https://twitter.com/Sillyy_Billyy/status/14441...,T
4,https://twitter.com/DendeCannabist/status/1380...,T


In [26]:
other_data.head(10)

,url,label
0,https://twitter.com/Arjun_fullBag/status/15217...,F
1,https://twitter.com/BtheChangeUsek/status/1521...,F
2,https://twitter.com/Julius_Owalu/status/152215...,F
3,https://twitter.com/StBeesSchool/status/152215...,F
4,https://twitter.com/JoshuaMuscat11g/status/152...,F
5,https://twitter.com/KalaleSteve/status/1521749...,F
6,https://twitter.com/scottiemutt/status/1521676...,F
7,https://twitter.com/Kryptok8ted/status/1521839...,F
8,https://twitter.com/MrsGinauPeeg/status/152215...,F
9,https://twitter.com/KMitch32118/status/1521952...,F


In [29]:
main_data.shape[0] + other_data.shape[0]

112057